In [3]:
from scrapy.crawler import CrawlerProcess
from src.spiders.KonessoSpider import KonessoSpider

process = CrawlerProcess(settings={
    "FEEDS": {
        "scrapped_pages/output_konesso.csv": {"format": "csv"},
    },
})
process.crawl(KonessoSpider)
process.start()


2024-11-24 17:17:51 [scrapy.utils.log] INFO: Scrapy 2.11.2 started (bot: scrapybot)
2024-11-24 17:17:51 [scrapy.utils.log] INFO: Versions: lxml 5.3.0.0, libxml2 2.11.7, cssselect 1.2.0, parsel 1.9.1, w3lib 2.2.1, Twisted 24.10.0, Python 3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)], pyOpenSSL 24.2.1 (OpenSSL 3.3.2 3 Sep 2024), cryptography 43.0.3, Platform Windows-10-10.0.22631-SP0
2024-11-24 17:17:51 [scrapy.addons] INFO: Enabled addons:
[]
2024-11-24 17:17:51 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2024-11-24 17:17:51 [scrapy.extensions.telnet] INFO: Telnet Password: cbd76ca7e6cfbb56
2024-11-24 17:17:51 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats']
2024-11-24 17:17:51 [scrapy.crawler] INFO: Overridden settings:
{}
2024-11-24 17:17:51 

ReactorNotRestartable: 

In [5]:
import pandas as pd

konesso_df = pd.read_csv(r"scrapped_pages/output_konesso.csv", delimiter=",")
konesso_df.head()

,name,price_old,price_netto,price_new
0,Kawa ziarnista Cornella Barista PRO Premium Gr...,"129,99 zł","89,83 zł +23% VAT","110,49 zł"
1,ZESTAW - Kawa ziarnista Costa Coffee Crema Ble...,NaN,"95,59 zł +23% VAT","117,58 zł"
2,Kawa ziarnista MRC. Cappuccino Blend 1kg,"120,00 zł","73,17 zł +23% VAT","90,00 zł"
3,ZESTAW - Kawa ziarnista Bazzara AROMAMORE 2x1k...,NaN,"203,24 zł +23% VAT","249,99 zł"
4,ZESTAW - Kawa ziarnista Bazzara DOLCEVIVACE 2x...,NaN,"162,59 zł +23% VAT","199,98 zł"


In [7]:
konesso_df.dropna(inplace=True)
print(konesso_df)

                                                   name  price_old  \
0     Kawa ziarnista Cornella Barista PRO Premium Gr...  129,99 zł   
2              Kawa ziarnista MRC. Cappuccino Blend 1kg  120,00 zł   
5                  Kawa ziarnista Bazzara AROMAMORE 1kg  132,99 zł   
6             Kawa ziarnista Bazzara GRANCAPPUCCINO 1kg  129,99 zł   
8      Kawa ziarnista LaCava Colombia Jairo Arcila 200g  119,00 zł   
...                                                 ...        ...   
2526       Kawa ziarnista Costa Coffee Crema Blend 500g   39,99 zł   
2530  Kawa ziarnista Story Hakuna Matata Rwanda Gake...   59,00 zł   
2531   Kawa ziarnista Story Espresso Into The Wind 250g   39,00 zł   
2538  Kawa ziarnista Italcaffe Monorigine Colombia 250g   32,99 zł   
2539  Kawa ziarnista Italcaffe Espresso Italiano Roy...   99,99 zł   

            price_netto  price_new  
0     89,83 zł +23% VAT  110,49 zł  
2     73,17 zł +23% VAT   90,00 zł  
5     89,74 zł +23% VAT  110,38 zł  
6     87,72

In [20]:
import re

def clean_price(column):
    cleaned_price = re.sub(r'[^\d,]', '', column)
    return cleaned_price


def extract_weight(product_name):
    match = re.search(r'(\d+([,.]\d+)?)\s?(kg|g)', product_name, re.IGNORECASE)
    if match:
        weight_value = match.group(1)
        weight_unit = match.group(3).lower()

        if weight_unit == 'g':
            weight_value = float(weight_value.replace(',', '.')) / 1000
        else:
            weight_value = float(weight_value.replace(',', '.'))

        return weight_value, product_name.replace(match.group(0), '').strip()
    return None, product_name


konesso_df["price_old"] = konesso_df["price_old"].apply(clean_price)
konesso_df["price_netto"] = konesso_df["price_netto"].apply(clean_price)
konesso_df["price_new"] = konesso_df["price_new"].apply(clean_price)
konesso_df["package_weight [KG]"], konesso_df["name"] =  zip(*konesso_df['name'].apply(extract_weight))

konesso_df.rename(columns={
    "name": "product_name",
    "price_old": "price_old [PLN]",
    "price_netto": "price_netto [PLN]",
    "price_new": "price_new [PLN]",
}, inplace=True)

konesso_df.head()

,product_name,price_old [PLN],price_netto [PLN],price_new [PLN],package_weight [KG]
0,Kawa ziarnista Cornella Barista PRO Premium Gr...,"129,99","89,8323","110,49",1.0
2,Kawa ziarnista MRC. Cappuccino Blend,"120,00","73,1723","90,00",1.0
5,Kawa ziarnista Bazzara AROMAMORE,"132,99","89,7423","110,38",1.0
6,Kawa ziarnista Bazzara GRANCAPPUCCINO,"129,99","87,7223","107,89",1.0
8,Kawa ziarnista LaCava Colombia Jairo Arcila,"119,00","67,7223","83,30",0.2


In [23]:
konesso_df.to_csv(r'cleaned_data/konesso_cleaned.csv', index=False, encoding='utf-8')